In [4]:
using StartUpDG, GLMakie, Plots
using Triangulate

# dependencies
using GeometryBasics
using LinearAlgebra

N = 3
Nplot = 3
elementType = Quad()
rd = RefElemData(elementType,N,Nplot=N)
md = MeshData(uniform_mesh(elementType,100)...,rd)

@unpack x,y = md
x = @. x + .125*cos(pi/2 * (x-.5))*cos(3*pi/2*(y-.7))
y = @. y + .125*sin(4*pi/2 * (x-.75))*cos(pi/2*(y-.8))

xq,yq = rd.Vq*x,rd.Vq*y
u = @. sin(pi*x)*sin(pi*y)*exp(-5*(x^2+y^2))
u[:,1:md.K÷2] .+= .25
# u = rd.Pq*(@. xq + yq > 0)

Nfplot = Nplot+1
rfp,sfp = StartUpDG.map_face_nodes(elementType, LinRange(-1,1,Nfplot))
Vfp = vandermonde(elementType,N,rfp,sfp)/rd.VDM
xf,yf = (x->Vfp*x).((x,y)) 

# break line segments up with NaNs
xfp,yfp,ufp = (xf->vec(vcat(reshape(xf,Nfplot,rd.Nfaces*md.K),fill(NaN,1,rd.Nfaces*md.K)))).((xf,yf,Vfp*u))

([-0.9861730161198451, -0.9796519007883802, -0.9731322857774334, -0.966614154961151, NaN, -0.966614154961151, -0.9692754373676234, -0.9719473034573364, -0.9746271164470561, NaN  …  1.0138269838801544, 1.0073043522585485, 1.0007801885600844, 0.9942544771577224, NaN, 0.9942544771577224, 0.9970743871668359, 0.9998774471032498, 1.0026608907125187, NaN], [-1.118433703551365, -1.1179117678236385, -1.117192101199402, -1.116276044731169, NaN, -1.116276044731169, -1.109603664630928, -1.1028847281483558, -1.0961186966489493, NaN  …  1.1184337035513645, 1.118756879468433, 1.1188806305982755, 1.1188046080090213, NaN, 1.1188046080090213, 1.1125932192217358, 1.1063315652484478, 1.1000199347339943, NaN], [0.24999976548022027, 0.2499996315685279, 0.24999948066666955, 0.24999931112154672, NaN, 0.24999931112154672, 0.24999937097060165, 0.24999943266072108, 0.2499994956320941, NaN  …  1.7786062802395516e-7, 1.0028256809997634e-7, 1.14346284264214e-8, -8.987385034225949e-8, NaN, -8.987385034225949e-8, -4.

In [ ]:
# plot makie mesh -- does not work!
Makie.mesh(rd.Vp*u,(x->rd.Vp*x).(rd.rst),(x->rd.Vp*x).((x,y)),color=vec(rd.Vp*u))
# draw mesh lines above/below
Makie.translate!(Makie.lines!(xfp,yfp,ufp,color=:white,linewidth=.5),0,0,1e-3) 
Makie.translate!(Makie.lines!(xfp,yfp,ufp,color=:white,linewidth=.5),0,0,-1e-3)
Makie.current_figure()